**INAIL SCRAPER**

AUTORE: Francesco Natali
DATA: Gennaio 2025
SCOPO: Scraping automatico e intelligente delle pubblicazioni INAIL per tesi

CARATTERISTICHE:
- ✅ Scraping incrementale (skippa documenti già processati)
- ✅ Estrazione automatica di PDF con Docling
- ✅ Analisi VLM delle immagini (SmolVLM)
- ✅ Estrazione strutturata di tabelle e figure
- ✅ Backup automatico su Google Drive
- ✅ Manifest JSON per Vector Database

REQUISITI SISTEMA:
- Google Colab (con GPU per VLM)
- Google Drive montato
- Connessione internet stabile

UTILIZZO:
1. Eseguire setup iniziale
2. Scegliere modalità di scraping
3. Lo script gestisce automaticamente backup e recovery

In [ ]:
# SETUP E INSTALLAZIONE DIPENDENZE

# Installa pacchetti necessari
# %pip install -q google-colab-selenium docling "docling[vlm]" transformers torch pillow accelerate

# Import librerie standard
import json
import re
import time
import random
import shutil
import unicodedata
from pathlib import Path
from datetime import datetime
from typing import Dict, Any, Optional, List
from urllib.parse import urljoin, quote_plus

# Import librerie per scraping
import google_colab_selenium as gs
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

In [ ]:
# CONFIGURAZIONE GLOBALE

class INAILConfig:

    # Configurazione centralizzata per lo scraper INAIL. Gestisce Directory di output (locale e Drive), Impostazioni VLM (Vision Language Model), Prompt per analisi immagini


    # Directory locali (su istanza Colab)
    OUTPUT_DIR = Path('./inail_scraped_data')
    PDF_DIR = OUTPUT_DIR / 'pdfs'
    JSON_DIR = OUTPUT_DIR / 'json'
    IMAGES_DIR = OUTPUT_DIR / 'images'
    TABLES_DIR = OUTPUT_DIR / 'tables'

    # Directory backup su Google Drive
    DRIVE_BACKUP_DIR = Path('/content/drive/MyDrive/INAIL_Thesis_Data')

    # Configurazione Vision Language Model
    VLM_ENABLED = True
    VLM_MODEL = "HuggingFaceTB/SmolVLM-256M-Instruct"
    VLM_PROMPT = """Describe this technical document image in detail for academic research. Focus on:
- Main content type (diagram, chart, photo, schematic)
- Key visual elements and their relationships
- Any visible text, labels, or numerical data
- Technical details relevant to workplace safety or industrial processes
Be precise and comprehensive."""

    @classmethod
    def setup_directories(cls):
        """Crea tutte le directory necessarie se non esistono"""
        for dir_path in [cls.OUTPUT_DIR, cls.PDF_DIR, cls.JSON_DIR,
                        cls.IMAGES_DIR, cls.TABLES_DIR]:
            dir_path.mkdir(exist_ok=True, parents=True)
        print(f"[INFO] ✅ Directory di output pronte: {cls.OUTPUT_DIR}")

In [ ]:
# INIZIALIZZAZIONE SELENIUM DRIVER

def initialize_selenium_driver():

    # Inizializza il browser Selenium con configurazione ottimizzata
    # Configurazioni Headless mode (nessuna finestra visibile), Timeout aumentati per stabilità, User agent realistico
    # Returns driver: Istanza di Chrome WebDriver

    print("[SETUP] Inizializzazione Selenium WebDriver...")

    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--disable-software-rasterizer')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

    driver = gs.Chrome(options=options)
    driver.set_page_load_timeout(180)  # 3 minuti per page load
    driver.implicitly_wait(20)  # Wait implicito 20 secondi

    print("[SETUP] ✅ Driver Selenium pronto\n")
    return driver

# Inizializza driver globale
driver = initialize_selenium_driver()
BASE_CATALOG_URL = "https://www.inail.it/portale/it/inail-comunica/pubblicazioni/catalogo-generale.html"


In [ ]:
# GOOGLE DRIVE - BACKUP E RIPRISTINO

def mount_google_drive():

    # Monta Google Drive nell'ambiente Colab.
    # Necessario per backup persistente dei dati.

    try:
        from google.colab import drive
        if not Path('/content/drive').exists():
            drive.mount('/content/drive', force_remount=False)
            print("[SUCCESS] ✅ Google Drive montato")
        else:
            print("[INFO] Google Drive già montato")
        return True
    except Exception as e:
        print(f"[WARNING] ⚠️ Impossibile montare Drive: {e}")
        return False


def backup_to_drive():

    # Esegue backup completo di tutti i dati scrapati su Google Drive
    # Backup include PDF originali, JSON con metadati, Immagini estratte + descrizioni VLM, Tabelle estratte, Manifest per vector database
    # Returns bool: True se backup riuscito, False altrimenti

    source = INAILConfig.OUTPUT_DIR
    dest = INAILConfig.DRIVE_BACKUP_DIR

    if not source.exists():
        print("[ERROR] ❌ Nessun dato da backuppare")
        return False

    try:
        if not Path('/content/drive').exists():
            mount_google_drive()

        print(f"\n{'='*80}")
        print(f"BACKUP SU GOOGLE DRIVE")
        print(f"{'='*80}")
        print(f"Origine: {source}")
        print(f"Destinazione: {dest}\n")

        # Copia ricorsiva di tutti i file
        shutil.copytree(str(source), str(dest), dirs_exist_ok=True)

        # Statistiche backup
        total_files = sum(1 for _ in dest.rglob('*') if _.is_file())
        total_size_mb = sum(f.stat().st_size for f in dest.rglob('*') if f.is_file()) / (1024**2)

        print(f"✅ Backup completato!")
        print(f"📦 File totali: {total_files}")
        print(f"💾 Dimensione: {total_size_mb:.1f} MB")
        print(f"📁 Path Drive: {dest}")
        print(f"{'='*80}\n")

        return True
    except Exception as e:
        print(f"[ERROR] ❌ Backup fallito: {e}")
        return False


def restore_from_drive():
    # Ripristina dati da backup Drive
    # Returns bool: True se ripristino riuscito, False altrimenti

    source = INAILConfig.DRIVE_BACKUP_DIR
    dest = INAILConfig.OUTPUT_DIR

    try:
        if not Path('/content/drive').exists():
            mount_google_drive()

        if not source.exists():
            print("[INFO] ℹ️ Nessun backup trovato su Drive (prima esecuzione)")
            return False

        print(f"\n{'='*80}")
        print(f"RIPRISTINO DA GOOGLE DRIVE")
        print(f"{'='*80}")

        shutil.copytree(str(source), str(dest), dirs_exist_ok=True)

        total_files = sum(1 for _ in dest.rglob('*') if _.is_file())

        print(f"✅ Ripristino completato!")
        print(f"📦 File ripristinati: {total_files}")
        print(f"{'='*80}\n")

        return True
    except Exception as e:
        print(f"[ERROR] ❌ Ripristino fallito: {e}")
        return False

In [ ]:
# DOCLING - PROCESSAMENTO PDF

def initialize_docling_converter():

    # Inizializza Docling per estrazione contenuti da PDF
    # Docling è lo stato dell'arte per Estrazione testo preservando struttura, Estrazione immagini ad alta risoluzione, OCR su documenti scansionati, Riconoscimento tabelle
    # Returns converter: Istanza di DocumentConverter

    from docling.datamodel.pipeline_options import PdfPipelineOptions
    from docling.datamodel.base_models import InputFormat
    from docling.document_converter import DocumentConverter, PdfFormatOption

    # Configurazione pipeline
    pipeline_options = PdfPipelineOptions()
    pipeline_options.generate_picture_images = True  # Estrai immagini
    pipeline_options.images_scale = 2.0  # Scala 2x per qualità
    pipeline_options.do_ocr = True  # OCR su immagini
    pipeline_options.do_picture_description = False  # Gestito da VLM

    converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    vlm_status = "ON" if INAILConfig.VLM_ENABLED else "OFF"
    print(f"[INFO] ✅ Docling pronto | VLM: {vlm_status}")

    return converter

In [ ]:
# HELPER FUNCTIONS - UTILITÀ GENERALI

def download_pdf_inail(pdf_url: str, output_path: Path) -> bool:

    # Scarica PDF da URL INAIL
    # Args: pdf_url URL del PDF da scaricare, output_path Path dove salvare il file
    # Returns: True se download riuscito

    import os
    try:
        os.system(f'wget -q -O {output_path} "{pdf_url}"')
        return output_path.exists() and output_path.stat().st_size > 0
    except:
        return False


def normalize_query(q: str) -> str:
    # Normalizza query di ricerca per URL encoding
    q = unicodedata.normalize("NFKC", q).strip().lower()
    return " ".join(q.split()).replace("'", "'")


def validate_date(date_str: str) -> bool:
    #Valida formato data dd/mm/yyyy
    return bool(re.match(r"^\d{2}/\d{2}/\d{4}$", date_str))


def build_inail_search_url(query: str = None, page: int = 1,
                          start_date: str = None, end_date: str = None) -> str:

    # Costruisce URL di ricerca catalogo INAIL
    # Args: query Termini di ricerca (opzionale), page Numero pagina (default: 1), start_date Data inizio in formato dd/mm/yyyy, end_date Data fine in formato dd/mm/yyyy
    # Returns: str URL completo per la ricerca

    if page < 1:
        raise ValueError("Pagina deve essere >= 1")

    params = []
    if query and query.strip():
        params.append(f"text={quote_plus(normalize_query(query))}")
    if start_date and validate_date(start_date):
        params.append(f"startDate={quote_plus(start_date)}")
    if end_date and validate_date(end_date):
        params.append(f"endDate={quote_plus(end_date)}")
    params.append(f"page={page}")

    return f"{BASE_CATALOG_URL}?{'&'.join(params)}"

In [ ]:
# DRIVER MANAGEMENT - GESTIONE CRASH SELENIUM

def recreate_driver():

    # Ricrea il driver Selenium in caso di timeout/disconnessione
    # Gestisce automaticamente Chiusura driver corrotto, Creazione nuovo driver, Attesa stabilizzazione
    # Returns: driver Nuovo driver Selenium

    global driver

    try:
        driver.quit()
    except:
        pass

    print("\n[INFO] ♻️  Ricreo driver Selenium...")

    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')

    driver = gs.Chrome(options=options)
    driver.set_page_load_timeout(180)
    driver.implicitly_wait(20)

    time.sleep(5)  # Stabilizzazione
    print("[INFO] ✅ Driver ricreato\n")

    return driver

In [ ]:
# LINK EXTRACTION - ESTRAZIONE PUBBLICAZIONI

def extract_cards_with_wait(driver, timeout: int = 20, max_retries: int = 4) -> List[Dict[str, str]]:

    # Estrae card delle pubblicazioni da una pagina di risultati
    # Gestisce: Retry automatici su timeout, Refresh pagina in caso di errori, Parsing robusto dell'HTML
    # Args: driver Istanza Selenium WebDriver, timeout Timeout in secondi (default: 20), max_retries Numero massimo di tentativi (default: 4)
    # Returns: Lista di pubblicazioni trovate [{titolo, url}, ...]

    for attempt in range(max_retries):
        try:
            print(f"    [Tentativo {attempt + 1}/{max_retries}]", end=" ")

            # Attendi caricamento elementi
            WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h3.card-title a[href], body"))
            )

            time.sleep(3)  # Pausa per rendering JavaScript

            # Estrai card
            results = []
            cards_elements = driver.find_elements(By.CSS_SELECTOR, "h3.card-title a[href]")

            for a in cards_elements:
                try:
                    href = a.get_attribute("href")
                    text = a.text.strip()
                    if href and text:
                        results.append({
                            "titolo": text,
                            "url": urljoin("https://www.inail.it", href)
                        })
                except:
                    continue

            if results:
                print(f"✓ {len(results)} card")
                return results
            else:
                print("✗ Nessuna card")

        except TimeoutException:
            print(f"✗ Timeout")
            if attempt < max_retries - 1:
                wait_time = random.uniform(10, 15)
                print(f"    [Pausa {wait_time:.1f}s]")
                time.sleep(wait_time)
                try:
                    driver.refresh()
                    time.sleep(5)
                except:
                    pass

        except Exception as e:
            print(f"✗ Errore: {str(e)[:50]}")
            if attempt < max_retries - 1:
                time.sleep(8)

    print("    [FALLITO]")
    return []


def get_inail_publication_links(driver, query: str = None, max_pages: int = 1,
                               start_date: str = None, end_date: str = None) -> List[Dict[str, str]]:

    # Estrae link di tutte le pubblicazioni INAIL da più pagine di risultati.
    # Features: Stop intelligente su pagine vuote consecutive, Auto-retry su timeout/connection errors, Ricreazione automatica driver se necessario, Deduplicazione link
    # Args: driver Istanza Selenium WebDriver, query Termini di ricerca (opzionale), max_pages Numero massimo di pagine da esplorare, end_date: Data fine filtraggio (dd/mm/yyyy)
    # Returns: Lista completa pubblicazioni trovate

    all_links = []
    seen = set()
    consecutive_failures = 0
    MAX_FAILURES = 2

    print(f"\n{'='*80}")
    print(f"ESTRAZIONE LINK PUBBLICAZIONI")
    print(f"{'='*80}")
    print(f"Query: {query if query else '(tutte)'} | Max pagine: {max_pages}\n")

    for page_num in range(1, max_pages + 1):
        url = build_inail_search_url(query=query, page=page_num,
                                     start_date=start_date, end_date=end_date)

        print(f"[Pagina {page_num}/{max_pages}]")

        try:
            driver.get(url)

            # Pausa tra pagine per evitare rate limiting
            if page_num > 1:
                delay = random.uniform(6, 10)
                print(f"  [Pausa {delay:.1f}s]")
                time.sleep(delay)

            # Estrai card da pagina
            cards = extract_cards_with_wait(driver, timeout=20)

            if not cards:
                consecutive_failures += 1
                print(f"  ⚠️ Pagina vuota ({consecutive_failures}/{MAX_FAILURES})")

                if consecutive_failures >= MAX_FAILURES:
                    print(f"\n  🛑 STOP: Troppe pagine vuote consecutive")
                    break

                time.sleep(random.uniform(10, 15))
                continue

            # Reset contatore failures se trovate card
            consecutive_failures = 0
            new_count = 0

            # Aggiungi card non duplicate
            for card in cards:
                if card["url"] not in seen:
                    seen.add(card["url"])
                    all_links.append(card)
                    new_count += 1

            print(f"  ✅ +{new_count} nuove (totale: {len(all_links)})")
            time.sleep(random.uniform(2, 4))

        except KeyboardInterrupt:
            print(f"\n⚠️ INTERRUZIONE MANUALE")
            break

        except Exception as e:
            consecutive_failures += 1
            error_msg = str(e)
            print(f"  ❌ Errore: {error_msg[:100]}")

            # Gestione errori di connessione/timeout
            if any(keyword in error_msg.lower() for keyword in
                   ['timeout', 'connection', 'disconnected', 'chrome']):
                print(f"  ⚠️  Problema con driver, ricreo...")
                try:
                    driver = recreate_driver()
                    consecutive_failures = 0
                    print(f"  ♻️  Ritento pagina {page_num}...")
                    continue
                except Exception as recreate_error:
                    print(f"  ❌ Impossibile ricreare driver: {recreate_error}")
                    break

            if consecutive_failures >= MAX_FAILURES:
                print(f"\n  🛑 STOP: Troppi errori consecutivi")
                break

            time.sleep(random.uniform(12, 18))

    print(f"\n{'='*80}")
    print(f"📊 Totale pubblicazioni trovate: {len(all_links)}")
    print(f"{'='*80}\n")

    return all_links

In [ ]:
# DOCUMENT PROCESSING - ANALISI STRUTTURA DOCUMENTI

def analyze_document_structure(doc) -> Dict[str, Any]:

    # Analizza struttura del documento PDF estratto da Docling
    # Estrae: Headings (titoli di sezione) con livello gerarchico, Tabelle con metadati (caption, colonne, righe), Figure con caption e posizione
    # Args: Documento Docling processato
    # Returns: Dict Struttura completa del documento

    structure = {
        'num_tables': 0,
        'num_figures': 0,
        'num_headings': 0,
        'headings': [],
        'tables': [],
        'figures': []
    }

    try:
        markdown = doc.export_to_markdown()
        lines = markdown.split('\n')

        # ESTRAZIONE HEADINGS
        for i, line in enumerate(lines):
            if line.strip().startswith('#'):
                level = len(line) - len(line.lstrip('#'))
                text = line.lstrip('#').strip()
                structure['num_headings'] += 1
                structure['headings'].append({
                    'type': f'heading_level_{level}',
                    'text': text[:200],  # Limita lunghezza
                    'position': i,
                    'level': level
                })

        # ESTRAZIONE TABELLE
        in_table = False
        current_table_lines = []
        table_start_idx = -1

        for i, line in enumerate(lines):
            # Rileva inizio tabella (linea con pipe e trattini)
            if '|' in line and ('-' in line or '─' in line):
                if not in_table:
                    in_table = True
                    table_start_idx = i
                    current_table_lines = []

            # Accumula righe tabella
            if in_table:
                if '|' in line:
                    current_table_lines.append(line)
                else:
                    # Fine tabella
                    if current_table_lines:
                        # Cerca caption nelle righe precedenti
                        caption = 'N/A'
                        for j in range(max(0, table_start_idx-5), table_start_idx):
                            candidate = lines[j].strip()
                            if candidate and 'Tabella' in candidate:
                                caption = candidate
                                break

                        # Estrai colonne dalla prima riga
                        potential_columns = []
                        if current_table_lines:
                            first_row = current_table_lines[0]
                            cols = [c.strip() for c in first_row.split('|') if c.strip()]
                            potential_columns = [col for col in cols
                                               if col and col not in ['-', '─']]

                        structure['num_tables'] += 1
                        structure['tables'].append({
                            'table_id': f'table_{structure["num_tables"]}',
                            'caption': caption,
                            'text_content': '\n'.join(current_table_lines),
                            'num_rows': len(current_table_lines),
                            'num_columns': len(potential_columns),
                            'potential_columns': potential_columns,
                            'position': table_start_idx
                        })

                    in_table = False
                    current_table_lines = []

        # ESTRAZIONE FIGURE
        for i, line in enumerate(lines):
            if '<!-- image -->' in line.lower() or \
               (line.strip().startswith('![') and '](' in line):
                structure['num_figures'] += 1

                # Cerca caption nelle righe successive
                caption = 'N/A'
                for j in range(i+1, min(len(lines), i+5)):
                    candidate = lines[j].strip()
                    if candidate and not candidate.startswith('#'):
                        caption = candidate
                        break

                structure['figures'].append({
                    'figure_id': f'figure_{structure["num_figures"]}',
                    'caption': caption,
                    'position': i,
                    'vlm_description': None  # Verrà popolato da VLM
                })

    except Exception as e:
        print(f"[!] Errore analisi struttura: {e}")

    return structure


def export_images_from_document(doc, doc_id: str) -> List[Dict[str, Any]]:

    # Estrae e salva tutte le immagini da un documento Docling.
    # Args: doc Documento Docling processato, doc_id Identificativo univoco documento
    # Returns: Lista metadati immagini esportate

    images_dir = INAILConfig.IMAGES_DIR / doc_id
    images_dir.mkdir(parents=True, exist_ok=True)
    exported_images = []

    try:
        from docling_core.types.doc import PictureItem

        if hasattr(doc, 'iterate_items'):
            for element, _level in doc.iterate_items():
                if isinstance(element, PictureItem):
                    picture_counter = len(exported_images) + 1
                    img_filename = f"figure_{picture_counter}.png"
                    img_path = images_dir / img_filename

                    try:
                        pil_image = element.get_image(doc)
                        pil_image.save(str(img_path), "PNG")

                        exported_images.append({
                            'path': str(img_path),
                            'filename': img_filename,
                            'caption': getattr(element, 'caption', 'N/A'),
                            'position': len(exported_images),
                            'vlm_description': None  # Popolato dopo
                        })

                        print(f"  [+] Immagine salvata: {img_filename}")
                    except Exception as e:
                        print(f"  [!] Errore salvataggio figura: {e}")

        print(f"[OK] {len(exported_images)} immagini esportate")

    except Exception as e:
        print(f"[ERROR] Export immagini fallito: {e}")

    return exported_images


def export_tables_to_files(tables: List[Dict], doc_id: str) -> List[str]:

    # Salva tabelle estratte come file di testo con metadati
    # Ogni tabella viene salvata come file .txt contenente Caption e metadati, Colonne identificate, Contenuto completo della tabella
    # Args: tables Lista tabelle estratte, doc_id Identificativo univoco documento
    # Returns: Path dei file tabelle creati

    if not tables:
        return []

    tables_dir = INAILConfig.TABLES_DIR / doc_id
    tables_dir.mkdir(parents=True, exist_ok=True)
    exported_tables = []

    for table in tables:
        table_id = table['table_id']
        txt_path = tables_dir / f"{table_id}.txt"

        try:
            with open(txt_path, 'w', encoding='utf-8') as f:
                f.write("="*80 + "\n")
                f.write(f"TABELLA: {table_id}\n")
                f.write("="*80 + "\n\n")
                f.write(f"CAPTION: {table.get('caption', 'N/A')}\n\n")

                if table.get('potential_columns'):
                    f.write("COLONNE IDENTIFICATE:\n")
                    for idx, col in enumerate(table['potential_columns'], 1):
                        f.write(f"  {idx}. {col}\n")
                    f.write("\n")

                f.write("METADATI STRUTTURALI:\n")
                f.write(f"  - Righe: {table.get('num_rows', 'N/A')}\n")
                f.write(f"  - Colonne: {len(table.get('potential_columns', []))}\n")
                f.write(f"  - Posizione: {table.get('position', 'N/A')}\n\n")

                f.write("CONTENUTO:\n" + "-"*80 + "\n")
                f.write(table['text_content'])
                f.write("\n" + "-"*80 + "\n")

            exported_tables.append(str(txt_path))
            print(f"  [+] Tabella salvata: {txt_path.name}")

        except Exception as e:
            print(f"  [!] Errore salvataggio tabella: {e}")

    return exported_tables

In [ ]:
# VLM PROCESSING - ANALISI IMMAGINI CON VISION LANGUAGE MODEL

def add_vlm_descriptions_to_images(exported_images: List[Dict], doc_id: str) -> List[Dict]:

    # Genera descrizioni testuali delle immagini usando Vision Language Model.
    # Processo:
    # 1) Carica modello SmolVLM (256M parametri, ottimizzato per Colab)
    # 2) Per ogni immagine, genera descrizione dettagliata
    # 3) Salva descrizione sia nel JSON che in file .txt separato

    # IMPORTANTE per Vector Database Ogni immagine ha un file .txt con la descrizione VLM, Permette indicizzazione semantica delle immagini

    # Args: exported_images Lista metadati immagini, doc_id Identificativo documento
    # Returns: Lista aggiornata con descrizioni VLM

    if not exported_images or not INAILConfig.VLM_ENABLED:
        return exported_images

    try:
        from transformers import AutoProcessor, AutoModelForVision2Seq
        from PIL import Image
        import torch

        print(f"\n[VLM] Caricamento modello {INAILConfig.VLM_MODEL}...")

        # Carica processore e modello
        processor = AutoProcessor.from_pretrained(
            INAILConfig.VLM_MODEL,
            trust_remote_code=True
        )
        model = AutoModelForVision2Seq.from_pretrained(
            INAILConfig.VLM_MODEL,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto",
            trust_remote_code=True
        )

        # Processa ogni immagine
        for idx, img_data in enumerate(exported_images, 1):
            try:
                print(f"  [{idx}/{len(exported_images)}] Analisi: {img_data['filename']}")

                # Carica immagine
                image = Image.open(img_data['path']).convert('RGB')

                # Prepara prompt per VLM
                messages = [{
                    "role": "user",
                    "content": [
                        {"type": "image"},
                        {"type": "text", "text": INAILConfig.VLM_PROMPT}
                    ]
                }]

                # Genera descrizione
                prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
                inputs = processor(text=prompt, images=[image], return_tensors="pt").to(model.device)

                with torch.no_grad():
                    outputs = model.generate(**inputs, max_new_tokens=300, do_sample=False)

                # Estrai descrizione dall'output
                full_output = processor.decode(outputs[0], skip_special_tokens=True)
                description = full_output.split("Assistant:")[-1].strip() \
                             if "Assistant:" in full_output else full_output.strip()

                # Salva nel dizionario
                img_data['vlm_description'] = description

                # SALVA FILE .TXT con descrizione VLM
                # Questo è CRUCIALE per il vector database!
                img_path = Path(img_data['path'])
                txt_path = img_path.with_suffix('.txt')

                with open(txt_path, 'w', encoding='utf-8') as f:
                    f.write("="*80 + "\n")
                    f.write(f"VLM DESCRIPTION - {img_data['filename']}\n")
                    f.write("="*80 + "\n\n")

                    f.write("IMAGE METADATA:\n")
                    f.write(f"  - Filename: {img_data['filename']}\n")
                    f.write(f"  - Caption: {img_data.get('caption', 'N/A')}\n")
                    f.write(f"  - Position: {img_data.get('position', 'N/A')}\n")
                    f.write(f"  - Model: {INAILConfig.VLM_MODEL}\n")
                    f.write(f"  - Timestamp: {datetime.now().isoformat()}\n\n")

                    f.write("VLM ANALYSIS:\n")
                    f.write("-"*80 + "\n")
                    f.write(description)
                    f.write("\n" + "-"*80 + "\n")

                img_data['vlm_description_file'] = str(txt_path)
                print(f"    ✓ Descrizione: {description[:60]}...")

            except Exception as e:
                print(f"    ✗ Errore: {e}")
                img_data['vlm_description'] = None
                img_data['vlm_description_file'] = None

        # Cleanup memoria
        del model, processor
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        success = sum(1 for img in exported_images if img.get('vlm_description'))
        print(f"[VLM] Completato: {success}/{len(exported_images)} descrizioni generate\n")

    except Exception as e:
        print(f"[ERROR] VLM fallito: {e}")

    return exported_images

In [ ]:
# MAIN SCRAPING - PROCESSAMENTO SINGOLA PUBBLICAZIONE

def scrape_inail_publication(driver, publication_url: str, converter,
                             save_pdf: bool = True,
                             enable_vlm: Optional[bool] = None) -> Dict[str, Any]:

    # Scrapa una singola pubblicazione INAIL completa.

    # Pipeline completa:
    # 1) Estrae metadati dalla pagina web (titolo, data, abstract)
    # 2) Scarica PDF
    # 3) Processa PDF con Docling (testo, tabelle, immagini)
    # 4) Analizza struttura documento
    # 5) Genera descrizioni VLM per immagini
    # 6) Salva tutto in JSON strutturato

    # Args: driver Selenium WebDriver, publication_url URL della pubblicazione, converter Docling converter, save_pdf Se True, mantiene PDF scaricato, enable_vlm Abilita/disabilita VLM (None = usa default config)
    # Returns: Dati completi pubblicazione processata

    use_vlm = enable_vlm if enable_vlm is not None else INAILConfig.VLM_ENABLED

    print(f"\n{'='*80}")
    print(f"SCRAPING PUBBLICAZIONE")
    print(f"{'='*80}\n")

    scraping_metadata = {
        'url': publication_url,
        'timestamp': datetime.now().isoformat(),
        'scraper_version': '1.0_THESIS',
        'docling_used': True,
        'vlm_enabled': use_vlm
    }

    try:
        # STEP 1: Estrai metadati web
        driver.get(publication_url)
        time.sleep(3)

        page = BeautifulSoup(driver.page_source, "lxml")

        # Titolo
        title_elem = page.find("h2", class_="h1")
        title = title_elem.get_text(strip=True) if title_elem else "N/A"

        # Abstract/Descrizione
        descr_blocks = page.find_all("p", class_="text-20")
        abstract = descr_blocks[1].get_text(strip=True) if len(descr_blocks) > 1 else ""

        # Data pubblicazione
        data_elem = page.find("strong", class_="js-date-value")
        data_pub = data_elem.get_text(strip=True).split(", ")[0] if data_elem else "N/A"

        # Link PDF
        link_pdf = page.find("ul", class_="list-download")
        pdf_url = None
        if link_pdf:
            a_tag = link_pdf.find("a", href=True)
            if a_tag:
                pdf_url = urljoin("https://www.inail.it", a_tag["href"])

        print(f"[+] Titolo: {title[:60]}...")
        print(f"[+] Data: {data_pub}")
        print(f"[+] PDF: {'Disponibile' if pdf_url else 'Non trovato'}")

        # STEP 2: Processa PDF
        pdf_data = None
        pdf_local_path = None

        if pdf_url:
            # Genera nome file sicuro
            safe_filename = re.sub(r'[^\w\-_.]', '_', title[:50]) + '.pdf'
            pdf_local_path = INAILConfig.PDF_DIR / safe_filename
            doc_id = safe_filename.replace('.pdf', '')

            print(f"\n[INFO] Download e processing PDF...")

            if download_pdf_inail(pdf_url, pdf_local_path):
                # Converti PDF con Docling
                result = converter.convert(str(pdf_local_path))
                doc = result.document

                # Esporta formati
                markdown_text = doc.export_to_markdown()
                plain_text = doc.export_to_text()

                # Analizza struttura
                structure_info = analyze_document_structure(doc)

                # Estrai immagini
                exported_images = export_images_from_document(doc, doc_id)

                # VLM su immagini
                if use_vlm and exported_images:
                    exported_images = add_vlm_descriptions_to_images(exported_images, doc_id)

                    # Aggiorna figure info con descrizioni VLM
                    for idx, fig_info in enumerate(structure_info.get('figures', [])):
                        if idx < len(exported_images):
                            vlm_desc = exported_images[idx].get('vlm_description')
                            if vlm_desc:
                                fig_info['vlm_description'] = vlm_desc

                # Esporta tabelle
                exported_tables = export_tables_to_files(structure_info['tables'], doc_id)

                # Statistiche
                num_pages = len(doc.pages) if hasattr(doc, 'pages') else 'N/A'
                print(f"\n[OK] Processamento completato:")
                print(f"  - Pagine: {num_pages}")
                print(f"  - Tabelle: {structure_info['num_tables']}")
                print(f"  - Figure: {structure_info['num_figures']}")
                print(f"  - Headings: {structure_info['num_headings']}")

                # Prepara dati per JSON
                pdf_data = {
                    'markdown_content': markdown_text,
                    'plain_text': plain_text,
                    'num_pages': num_pages,
                    'num_tables': structure_info['num_tables'],
                    'num_figures': structure_info['num_figures'],
                    'num_headings': structure_info['num_headings'],
                    'headings': structure_info['headings'],
                    'tables': structure_info['tables'],
                    'figures': structure_info['figures'],
                    'exported_images': exported_images,
                    'exported_tables': exported_tables
                }

                # Rimuovi PDF se non richiesto
                if not save_pdf:
                    pdf_local_path.unlink()
                    pdf_local_path = None

        # STEP 3: Ritorna risultato
        return {
            'scraping_metadata': scraping_metadata,
            'web_metadata': {
                'title': title,
                'abstract': abstract,
                'data_pubblicazione': data_pub,
                'pdf_url': pdf_url,
                'pdf_local_path': str(pdf_local_path) if pdf_local_path else None
            },
            'document_content': pdf_data,
            'status': 'success',
            'has_pdf': pdf_url is not None,
            'pdf_processed': pdf_data is not None
        }

    except Exception as e:
        print(f"[ERROR] Scraping fallito: {e}")
        return {
            'scraping_metadata': scraping_metadata,
            'status': 'error',
            'error_message': str(e)
        }


def save_to_json(data: Dict[str, Any], output_name: Optional[str] = None) -> Path:

    # Salva dati pubblicazione in file JSON
    # Args: Dati da salvare, output_name Nome file (opzionale, auto-generato se None)
    # Returns: Path del file JSON creato

    if output_name is None:
        title = data.get('web_metadata', {}).get('title', 'unknown')
        safe_title = re.sub(r'[^\w\-_.]', '_', title[:50])
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_name = f"{safe_title}_{timestamp}.json"

    output_path = INAILConfig.JSON_DIR / output_name

    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    return output_path

In [ ]:
# SCRAPING INCREMENTALE - SKIP DOCUMENTI GIÀ PROCESSATI

def load_already_scraped_urls() -> set:

    # Carica URL di tutte le pubblicazioni già processate.
    # Cerca in File JSON locali e Backup su Drive (se disponibile)
    # Questo permette scraping incrementale: nuove esecuzioni skippano automaticamente documenti già scaricati
    # Returns: set Set di URL già processati

    scraped_urls = set()

    # Directory dove cercare JSON
    search_dirs = [INAILConfig.JSON_DIR]

    if INAILConfig.DRIVE_BACKUP_DIR.exists():
        drive_json_dir = INAILConfig.DRIVE_BACKUP_DIR / 'json'
        if drive_json_dir.exists():
            search_dirs.append(drive_json_dir)

    # Scansiona tutti i JSON
    for json_dir in search_dirs:
        if not json_dir.exists():
            continue

        json_files = list(json_dir.glob("*.json"))
        json_files = [f for f in json_files if not f.name.startswith('vector_db_manifest')]

        for json_file in json_files:
            try:
                with open(json_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                # Gestisci sia file singoli che batch
                if isinstance(data, list):
                    docs = data
                else:
                    docs = [data]

                # Estrai URL
                for doc in docs:
                    url = doc.get('scraping_metadata', {}).get('url')
                    if url:
                        scraped_urls.add(url)
            except:
                continue

    return scraped_urls


def batch_scrape_inail_publications_incremental(
    driver, converter,
    query: str = None,
    max_pages: int = 5,
    max_documents: Optional[int] = None,
    enable_vlm: bool = False,
    save_pdfs: bool = True,
    delay_between_docs: int = 4,
    auto_backup_every: int = 5
) -> List[Dict[str, Any]]:

    # Scraping batch incrementale di pubblicazioni INAIL
    # Caratteristiche chiave: Skip automatico documenti già processati, Backup automatico ogni N documenti, Gestione interruzioni, Statistiche in tempo reale

    # Args:
    #    driver: Selenium WebDriver
    #    converter: Docling converter
    #    query: Query di ricerca (None = tutte)
    #    max_pages: Numero massimo pagine da esplorare
    #    max_documents: Limite documenti da processare (None = tutti)
    #    enable_vlm: Abilita analisi VLM immagini
    #    save_pdfs: Mantieni PDF scaricati
    #    delay_between_docs: Secondi tra documenti
    #   auto_backup_every: Backup ogni N documenti processati

    # Returns: Lista documenti processati con successo


    # STEP 1: Carica documenti già fatti
    already_scraped = load_already_scraped_urls()

    print(f"\n{'='*80}")
    print(f"SCRAPING INCREMENTALE")
    print(f"{'='*80}")
    print(f"📊 Documenti già processati: {len(already_scraped)}")
    print(f"{'='*80}\n")

    # STEP 2: Estrai link pubblicazioni
    publication_links = get_inail_publication_links(driver, query=query, max_pages=max_pages)

    if not publication_links:
        print("[WARNING] Nessuna pubblicazione trovata")
        return []

    # STEP 3: Filtra solo documenti nuovi
    new_links = [pub for pub in publication_links if pub['url'] not in already_scraped]

    print(f"\n{'='*80}")
    print(f"FILTRO DOCUMENTI")
    print(f"{'='*80}")
    print(f"📄 Totali trovati: {len(publication_links)}")
    print(f"⏭️  Già processati: {len(publication_links) - len(new_links)}")
    print(f"🆕 Nuovi da processare: {len(new_links)}")
    print(f"{'='*80}\n")

    if not new_links:
        print("✅ Tutti i documenti sono già stati processati!")
        return []

    # Limita numero documenti se richiesto
    if max_documents:
        new_links = new_links[:max_documents]
        print(f"[INFO] Limitato a {max_documents} nuovi documenti\n")

    # STEP 4: Scraping documenti nuovi
    results = []
    failed = []

    print(f"{'='*80}")
    print(f"SCRAPING {len(new_links)} NUOVI DOCUMENTI")
    print(f"{'='*80}\n")

    for idx, pub in enumerate(new_links, 1):
        print(f"\n[{idx}/{len(new_links)}] {pub['titolo'][:50]}...")

        try:
            # Scrapa pubblicazione
            result = scrape_inail_publication(
                driver=driver,
                publication_url=pub['url'],
                converter=converter,
                save_pdf=save_pdfs,
                enable_vlm=enable_vlm
            )

            if result['status'] == 'success':
                results.append(result)
                save_to_json(result)
                print(f"  ✅ Successo")

                # Backup automatico periodico
                if idx % auto_backup_every == 0:
                    print(f"\n  ☁️  Auto-backup ({idx}/{len(new_links)})...")
                    backup_to_drive()
            else:
                failed.append({
                    'url': pub['url'],
                    'title': pub['titolo'],
                    'error': result.get('error_message', 'Unknown')
                })
                print(f"  ❌ Fallito")

        except KeyboardInterrupt:
            print(f"\n⚠️ INTERRUZIONE MANUALE")
            print(f"💾 Backup dati salvati...")
            backup_to_drive()
            break

        except Exception as e:
            print(f"  ❌ Eccezione: {str(e)[:80]}")
            failed.append({
                'url': pub['url'],
                'title': pub['titolo'],
                'error': str(e)
            })

        # Pausa tra documenti
        if idx < len(new_links):
            delay = random.uniform(delay_between_docs, delay_between_docs + 2)
            time.sleep(delay)

    # STEP 5: Backup finale
    if results:
        print(f"\n☁️  Backup finale su Drive...")
        backup_to_drive()

    # STEP 6: Statistiche finali
    print(f"\n{'='*80}")
    print(f"SCRAPING COMPLETATO")
    print(f"{'='*80}")
    print(f"✅ Nuovi documenti: {len(results)}")
    print(f"❌ Falliti: {len(failed)}")
    print(f"📊 Totale documenti: {len(already_scraped) + len(results)}")
    print(f"{'='*80}\n")

    return results

In [ ]:
# VECTOR DATABASE - MANIFEST GENERATION

def create_vector_db_manifest(json_dir: Path = None) -> Path:

    # Crea manifest JSON aggregato per indicizzazione in vector database
    # Il manifest organizza TUTTI i contenuti testuali per l'embedding: Testi markdown completi, Tabelle con metadati (colonne, caption), Immagini con descrizioni VLM

    # Struttura manifest:
    # {
    #  "documents": [
    #    {
    #      "document_id": "...",
    #      "title": "...",
    #      "indexable_content": [
    #        {"type": "markdown_text", "content": "...", "metadata": {...}},
    #        {"type": "table", "content": "...", "metadata": {...}},
    #        {"type": "image_vlm", "content": "...", "metadata": {...}}
    #      ]
    #   }
    #  ]
    # }

    # Args: json_dir Directory JSON (default: INAILConfig.JSON_DIR)
    # Returns: Path del manifest creato

    if json_dir is None:
        json_dir = INAILConfig.JSON_DIR

    manifest = {
        'created_at': datetime.now().isoformat(),
        'purpose': 'Vector database indexing manifest',
        'version': '1.0',
        'total_documents': 0,
        'documents': []
    }

    json_files = list(json_dir.glob("*.json"))
    json_files = [f for f in json_files if not f.name.startswith('vector_db_manifest')]

    print(f"\n{'='*80}")
    print(f"CREAZIONE MANIFEST PER VECTOR DB")
    print(f"{'='*80}\n")

    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Gestisci sia file singoli che batch
            if isinstance(data, list):
                docs_to_process = data
            else:
                docs_to_process = [data]

            for doc in docs_to_process:
                if doc.get('status') != 'success':
                    continue

                doc_content = doc.get('document_content')
                if not doc_content:
                    continue

                web_meta = doc.get('web_metadata', {})
                doc_id = re.sub(r'[^\w\-_.]', '_', web_meta.get('title', 'unknown')[:50])

                # Entry per vector DB
                vector_entry = {
                    'document_id': doc_id,
                    'title': web_meta.get('title', 'N/A'),
                    'publication_date': web_meta.get('data_pubblicazione', 'N/A'),
                    'source_url': doc.get('scraping_metadata', {}).get('url'),
                    'pdf_url': web_meta.get('pdf_url'),
                    'content_types': [],
                    'indexable_content': []
                }

                # 1) TESTO MARKDOWN
                if doc_content.get('markdown_content'):
                    vector_entry['content_types'].append('markdown_text')
                    vector_entry['indexable_content'].append({
                        'type': 'markdown_text',
                        'content': doc_content['markdown_content'],
                        'metadata': {
                            'num_pages': doc_content.get('num_pages', 'N/A'),
                            'num_headings': doc_content.get('num_headings', 0)
                        }
                    })

                # 2) TABELLE con metadati
                for table_info in doc_content.get('tables', []):
                    vector_entry['content_types'].append('table')

                    # Testo arricchito: caption + colonne + contenuto
                    table_text = f"TABELLA: {table_info.get('caption', 'Senza titolo')}\n\n"

                    if table_info.get('potential_columns'):
                        table_text += "COLONNE: " + ", ".join(table_info['potential_columns']) + "\n\n"

                    table_text += "CONTENUTO:\n" + table_info.get('text_content', '')

                    vector_entry['indexable_content'].append({
                        'type': 'table',
                        'content': table_text,
                        'metadata': {
                            'table_id': table_info.get('table_id'),
                            'caption': table_info.get('caption'),
                            'num_rows': table_info.get('num_rows'),
                            'num_columns': table_info.get('num_columns'),
                            'columns': table_info.get('potential_columns', []),
                            'position': table_info.get('position')
                        }
                    })

                # 3) IMMAGINI con descrizioni VLM
                for img_info in doc_content.get('exported_images', []):
                    if img_info.get('vlm_description'):
                        vector_entry['content_types'].append('image_vlm')

                        # Testo combinato: caption + descrizione VLM
                        img_text = f"IMMAGINE: {img_info.get('filename')}\n"
                        img_text += f"CAPTION: {img_info.get('caption', 'N/A')}\n\n"
                        img_text += f"DESCRIZIONE VISIVA:\n{img_info['vlm_description']}"

                        # Path file .txt con descrizione
                        img_path = Path(img_info.get('path', ''))
                        txt_path = img_path.with_suffix('.txt')
                        vlm_file = str(txt_path) if txt_path.exists() else None

                        vector_entry['indexable_content'].append({
                            'type': 'image_vlm',
                            'content': img_text,
                            'metadata': {
                                'filename': img_info.get('filename'),
                                'caption': img_info.get('caption'),
                                'vlm_description_file': vlm_file,
                                'image_path': img_info.get('path'),
                                'position': img_info.get('position')
                            }
                        })

                # Aggiungi al manifest se ha contenuto
                if vector_entry['indexable_content']:
                    manifest['documents'].append(vector_entry)
                    manifest['total_documents'] += 1
                    print(f"  ✓ {doc_id}: {len(vector_entry['indexable_content'])} chunks")

        except Exception as e:
            print(f"  ✗ Errore {json_file.name}: {e}")

    # Salva manifest
    manifest_path = json_dir / f"vector_db_manifest_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

    with open(manifest_path, 'w', encoding='utf-8') as f:
        json.dump(manifest, f, ensure_ascii=False, indent=2)

    # Statistiche
    total_chunks = sum(len(doc['indexable_content']) for doc in manifest['documents'])
    total_tables = sum(1 for doc in manifest['documents']
                      for chunk in doc['indexable_content']
                      if chunk['type'] == 'table')
    total_images = sum(1 for doc in manifest['documents']
                      for chunk in doc['indexable_content']
                      if chunk['type'] == 'image_vlm')
    total_markdown = sum(1 for doc in manifest['documents']
                        for chunk in doc['indexable_content']
                        if chunk['type'] == 'markdown_text')

    print(f"\n{'='*80}")
    print(f"MANIFEST CREATO")
    print(f"{'='*80}")
    print(f"📄 File: {manifest_path.name}")
    print(f"📊 Documenti: {manifest['total_documents']}")
    print(f"📦 Chunks totali: {total_chunks}")
    print(f"   - Testi markdown: {total_markdown}")
    print(f"   - Tabelle: {total_tables}")
    print(f"   - Immagini VLM: {total_images}")
    print(f"{'='*80}\n")

    return manifest_path

In [ ]:
# MAIN - INTERFACCIA UTENTE

def main():

    # Funzione principale con menu interattivo.
    # Gestisce: Setup iniziale (directory, Drive, Docling), Ripristino automatico da backup, Menu scelta modalità, Scraping incrementale, Generazione manifest

    print("\n" + "="*80)
    print("INAIL SCRAPER - VERSIONE FINALE PER TESI")
    print("="*80 + "\n")

    # STEP 1) Setup directories
    INAILConfig.setup_directories()

    # STEP 2) Google Drive + ripristino
    print("\n[STEP 1] Setup Google Drive...")
    if mount_google_drive():
        print("\n[STEP 2] Ripristino dati da backup Drive...")
        restore_from_drive()

    # STEP 3) Mostra statistiche documenti già processati
    already_scraped = load_already_scraped_urls()
    print(f"\n[INFO] 📊 Documenti già processati: {len(already_scraped)}")

    # STEP 4) Inizializza Docling
    print("\n[STEP 3] Inizializzazione Docling...")
    converter = initialize_docling_converter()

    # STEP 5) Menu scelta modalità
    print("\n" + "="*80)
    print("MODALITÀ DISPONIBILI:")
    print("="*80)
    print("1. 🚀 Scraping incrementale (skippa già processati)")
    print("2. 📊 Verifica stato (statistiche documenti)")
    print("3. 📋 Crea manifest per Vector DB")
    print("4. ☁️  Backup manuale su Drive")
    print("5. 🔄 Ripristino da Drive")
    print("="*80 + "\n")

    choice = input("Scegli modalità (1-5, default 1): ").strip() or "1"

    # MODALITÀ 2: Verifica Stato
    if choice == "2":
        print(f"\n{'='*80}")
        print(f"STATO ATTUALE SISTEMA")
        print(f"{'='*80}")
        print(f"📊 Documenti processati: {len(already_scraped)}")
        print(f"📁 Path locale: {INAILConfig.OUTPUT_DIR}")
        print(f"☁️  Path Drive: {INAILConfig.DRIVE_BACKUP_DIR}")

        # Conta file per tipo
        if INAILConfig.OUTPUT_DIR.exists():
            pdfs = len(list(INAILConfig.PDF_DIR.glob('*.pdf'))) if INAILConfig.PDF_DIR.exists() else 0
            jsons = len(list(INAILConfig.JSON_DIR.glob('*.json'))) if INAILConfig.JSON_DIR.exists() else 0
            images = sum(1 for _ in INAILConfig.IMAGES_DIR.rglob('*.png')) if INAILConfig.IMAGES_DIR.exists() else 0
            tables = sum(1 for _ in INAILConfig.TABLES_DIR.rglob('*.txt')) if INAILConfig.TABLES_DIR.exists() else 0

            print(f"\n📦 File locali:")
            print(f"   - PDF: {pdfs}")
            print(f"   - JSON: {jsons}")
            print(f"   - Immagini: {images}")
            print(f"   - Tabelle: {tables}")

        print(f"{'='*80}\n")
        return None

    # MODALITÀ 3: Crea Manifest
    elif choice == "3":
        manifest_path = create_vector_db_manifest()
        print(f"✅ Manifest creato: {manifest_path}")

        # Backup manifest su Drive
        backup_choice = input("\nBackup manifest su Drive? (y/n): ").strip().lower()
        if backup_choice == 'y':
            backup_to_drive()

        return None

    # MODALITÀ 4: Backup Manuale
    elif choice == "4":
        backup_to_drive()
        return None

    # MODALITÀ 5: Ripristino
    elif choice == "5":
        restore_from_drive()
        return None

    # MODALITÀ 1: Scraping Incrementale
    print("\n" + "="*80)
    print("CONFIGURAZIONE SCRAPING INCREMENTALE")
    print("="*80 + "\n")

    # Parametri ricerca
    query = input("🔍 Query ricerca (Enter = tutte le pubblicazioni): ").strip() or None

    try:
        max_pages = int(input("📄 Numero massimo pagine da esplorare (default 5): ").strip() or "5")
    except:
        max_pages = 5

    max_docs_input = input("📊 Limite documenti da processare (Enter = tutti): ").strip()
    max_docs = int(max_docs_input) if max_docs_input else None

    vlm_input = input("🤖 Abilitare analisi VLM immagini? (y/n, default y): ").strip().lower()
    enable_vlm = (vlm_input != 'n')

    # Riepilogo configurazione
    print(f"\n{'='*80}")
    print("RIEPILOGO CONFIGURAZIONE:")
    print(f"{'='*80}")
    print(f"🔍 Query: {query if query else '(tutte le pubblicazioni)'}")
    print(f"📄 Pagine da esplorare: {max_pages}")
    print(f"📊 Limite documenti: {max_docs if max_docs else '(tutti i nuovi)'}")
    print(f"🤖 Analisi VLM: {'✅ Abilitata' if enable_vlm else '❌ Disabilitata'}")
    print(f"⏭️  Documenti già processati: {len(already_scraped)} (verranno skippati)")
    print(f"{'='*80}\n")

    confirm = input("▶️  Avviare scraping? (y/n): ").strip().lower()
    if confirm != 'y':
        print("\n❌ Operazione annullata")
        return None

    # Esegui scraping incrementale
    results = batch_scrape_inail_publications_incremental(
        driver=driver,
        converter=converter,
        query=query,
        max_pages=max_pages,
        max_documents=max_docs,
        enable_vlm=enable_vlm,
        save_pdfs=True,
        delay_between_docs=4,
        auto_backup_every=5
    )

    return results

In [ ]:
# UTILITY FUNCTIONS - COMANDI RAPIDI

def quick_backup():
    # Comando rapido per backup su Drive
    mount_google_drive()
    backup_to_drive()

def quick_restore():
    # Comando rapido per ripristino da Drive
    mount_google_drive()
    restore_from_drive()

def verify_vector_db_readiness():

    # Verifica che tutti i file siano pronti per vector database
    # Controlla Ogni immagine ha il suo file .txt con descrizione VLM, Tabelle estratte correttamente, Manifest generato

    print(f"\n{'='*80}")
    print(f"VERIFICA PREPARAZIONE VECTOR DB")
    print(f"{'='*80}\n")

    issues = []

    # Verifica immagini + descrizioni VLM
    if INAILConfig.IMAGES_DIR.exists():
        for doc_dir in INAILConfig.IMAGES_DIR.iterdir():
            if doc_dir.is_dir():
                png_files = list(doc_dir.glob("*.png"))
                txt_files = list(doc_dir.glob("*.txt"))

                print(f"📁 {doc_dir.name}:")
                print(f"   - Immagini: {len(png_files)}")
                print(f"   - Descrizioni VLM: {len(txt_files)}")

                if len(png_files) != len(txt_files):
                    issues.append(f"{doc_dir.name}: {len(png_files)} immagini ma {len(txt_files)} descrizioni")
                    print(f"   ⚠️  MISMATCH!")
                else:
                    print(f"   ✅ OK")

    # Verifica tabelle
    if INAILConfig.TABLES_DIR.exists():
        total_tables = sum(1 for _ in INAILConfig.TABLES_DIR.rglob("*.txt"))
        print(f"\n📊 Tabelle estratte: {total_tables}")

    # Verifica JSON
    json_files = list(INAILConfig.JSON_DIR.glob("*.json")) if INAILConfig.JSON_DIR.exists() else []
    print(f"\n📄 File JSON: {len(json_files)}")

    # Verifica manifest
    manifest_files = list(INAILConfig.JSON_DIR.glob("vector_db_manifest*.json")) if INAILConfig.JSON_DIR.exists() else []
    if manifest_files:
        print(f"📋 Manifest Vector DB: ✅ {len(manifest_files)} file")
        latest = max(manifest_files, key=lambda p: p.stat().st_mtime)
        print(f"   Ultimo: {latest.name}")
    else:
        print(f"📋 Manifest Vector DB: ❌ Non trovato")
        print(f"   → Esegui: create_vector_db_manifest()")

    # Report finale
    if issues:
        print(f"\n⚠️  PROBLEMI RILEVATI:")
        for issue in issues:
            print(f"   - {issue}")
    else:
        print(f"\n✅ Tutto pronto per indicizzazione Vector Database!")

    print(f"{'='*80}\n")

    return len(issues) == 0

In [ ]:
# ESECUZIONE SCRIPT
if __name__ == "__main__":

    # Entry point dello script.
    # Esegue 1) Main() per setup e scraping 2) Report finale con statistiche 3) Suggerimenti prossimi passi

    # Esegui main
    results = main()

    # Report finale
    print("\n" + "="*80)
    print("REPORT FINALE ESECUZIONE")
    print("="*80)

    if results:
        print(f"✅ Nuovi documenti processati: {len(results)}")

        # Statistiche dettagliate
        total_images = sum(len(r.get('document_content', {}).get('exported_images', []))
                          for r in results)
        total_tables = sum(r.get('document_content', {}).get('num_tables', 0)
                          for r in results)

        print(f"📊 Statistiche sessione:")
        print(f"   - Immagini estratte: {total_images}")
        print(f"   - Tabelle estratte: {total_tables}")

    # Statistiche totali
    total_docs = len(load_already_scraped_urls())
    print(f"\n📊 Totale documenti nel dataset: {total_docs}")
    print(f"📁 Output locale: {INAILConfig.OUTPUT_DIR}")
    print(f"☁️  Backup Drive: {INAILConfig.DRIVE_BACKUP_DIR}")

    # Suggerimenti prossimi passi
    print(f"\n{'='*80}")
    print("PROSSIMI PASSI SUGGERITI:")
    print(f"{'='*80}")
    print("1. Verifica preparazione Vector DB:")
    print("   → verify_vector_db_readiness()")
    print("\n2. Genera manifest per indicizzazione:")
    print("   → create_vector_db_manifest()")
    print("\n3. Backup finale su Drive:")
    print("   → quick_backup()")
    print(f"{'='*80}\n")

    print("✅ Script completato con successo!")

In [ ]:
# ============================================================================
# DOCUMENTAZIONE FINALE
# ============================================================================

"""
================================================================================
GUIDA UTILIZZO PER TESI
================================================================================

SETUP INIZIALE:
1. Caricare questo script in Google Colab
2. Eseguire tutte le celle
3. Lo script monterà automaticamente Google Drive
4. I dati verranno salvati in: /content/drive/MyDrive/INAIL_Thesis_Data

MODALITÀ SCRAPING:
- Scraping incrementale: Skippa automaticamente documenti già processati
- VLM opzionale: Genera descrizioni testuali delle immagini
- Backup automatico: Salvataggio su Drive ogni 5 documenti

OUTPUT GENERATO:
1. PDF originali (opzionale)
2. JSON con metadati completi
3. Immagini estratte + descrizioni VLM (.txt)
4. Tabelle estratte (.txt)
5. Manifest per Vector Database

STRUTTURA DATI PER VECTOR DB:
vector_db_manifest_YYYYMMDD_HHMMSS.json
├── documents[]
    ├── document_id
    ├── title
    ├── indexable_content[]
        ├── type: "markdown_text" | "table" | "image_vlm"
        ├── content: testo completo
        └── metadata: metadati specifici

EMBEDDING E INDICIZZAZIONE:
1. Caricare manifest JSON
2. Per ogni chunk in indexable_content:
   - Generare embedding (OpenAI/Cohere/etc)
   - Inserire in Vector DB (ChromaDB/Pinecone/Weaviate)
3. Conservare metadata per retrieval

COMANDI UTILI:
- quick_backup(): Backup rapido su Drive
- quick_restore(): Ripristino da Drive
- verify_vector_db_readiness(): Verifica dati per Vector DB
- create_vector_db_manifest(): Genera manifest

TROUBLESHOOTING:
- Timeout Selenium: Lo script ricrea automaticamente il driver
- Interruzione (Ctrl+C): Backup automatico dati salvati
- Sessione Colab scaduta: Ri-eseguire, ripristino automatico da Drive

RIFERIMENTI:
- Docling: https://github.com/docling-project/docling
- SmolVLM: https://huggingface.co/HuggingFaceTB/SmolVLM-256M-Instruct
- INAIL Catalogo: https://www.inail.it/pubblicazioni

================================================================================
FINE SCRIPT
================================================================================
"""